### Part 1: The ”Contract” & Few-Shot Learning

In [30]:
import sys, os
sys.path.append('..') 
os.getcwd()

'c:\\Users\\user\\Downloads\\ZUU CREW\\AI Engineer Essentials\\Mini Projects\\Mini Project 00\\Mini Project 00 - Operation Ditwah_Crisis Intelligence Pipeline\\notebooks'

##### *Import Libraries*

In [31]:
import pandas as pd
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.router import pick_model
from utils.logging_utils import log_llm_call

##### *Load data  from Sample Messages.txt*

In [32]:
input_file_path = '../data/Sample Messages.txt'
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

try:
    with open(input_file_path, 'r') as f:
        messages = [line.strip() for line in f.readlines() if line.strip()]
    print(f"Loaded {len(messages)} messages.")

    print("First 5 messages:")
    for msg in messages[:5]:
        print(msg)
    
except FileNotFoundError:
    print(f"Error: File not found at {input_file_path}")
    messages = []

Loaded 50 messages.
First 5 messages:
BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.
SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.
Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.
Does anyone have extra dry rations for the camp in Gampaha?
News just in: Kelani river water level is at 7ft.


##### *Setup LLM Client*

In [33]:
model = pick_model("openai", "general")
client = LLMClient("openai", model)

##### *Examples for Rescue, Supply, Info, Other classifications*

In [34]:
examples = """
Example 1:
        Message: "We are stuck in our house with rising flood water up to waist level in Hanwella"
        Output: District: Colombo | Intent: Rescue | Priority: High

Example 2:
        Message: "Need baby formula and diapers for displaced families at the school shelter"
        Output: District: Kandy | Intent: Supply | Priority: High

Example 3:
        Message: "Warning: Moragahakanda Dam may breach within 2 hours. Immediate evacuation ordered for downstream Kalu river villages."
        Output: District: Matale | Intent: Info | Priority: High

Example 4:
        Message: "Heart attack patient in Batticaloa needs immediate ambulance. No transport available due to flooding and landsliding."
        Output: District: Batticaloa | Intent: Other | Priority: High

Example 5:
        Message: "Reporting an old landslide blockage in Kelanimulla from previous rains. Not blocking anything urgent"
        Output: District: Colombo | Intent: Rescue | Priority: Low

Example 6:
        Message: "For future preparedness along Attanagalu Oya, we need sandbags and tarpaulin sheets for storage."
        Output: District: Gampaha | Intent: Supply | Priority: Low

Example 7:
        Message: "Heavy rains expected to continue for 24 hours"
        Output: District: None | Intent: Info | Priority: Low

Example 8:
        Message: "Road from Kandy to Nuwara Eliya is now open after landslide clearance"
        Output: District: None | Intent: Other | Priority: Low

Example 9:
        Message: "Update: Gin river water levels rising rapidly."
        Output: District: Galle | Intent: Info | Priority: Low

Example 10:
        Message: "Kelani river has reached overflow levels."
        Output: District: Colombo | Intent: Info | Priority: Low
"""

##### *Process Loop*

In [42]:
results = []

print(f"{'District':<15} | {'Intent':<15} | {'Priority':<10} | {'Message':<50}")
print("-" * 80)

for msg in messages:
    prompt_text, spec = render(
                                'few_shot.v1',
                                role='Crisis Intelligence Classifier',
                                examples=examples,
                                query=f'Message: "{msg}"',
                                constraints='Classify the message exactly according to the Output format. Do not add extra text.',
                                format='District: [Name] | Intent: [Category] | Priority: [High/Low]'
                                )

    response = client.chat([{'role': 'user', 'content': prompt_text}], temperature=0.0)
    output_text = response['text'].strip()
    clean_text = output_text.replace("Output:", "").strip()
    
    parsed_data = {
                    'original_message': msg, 
                    'raw_output': output_text,
                    'district': 'Unknown',
                    'intent': 'Unknown',
                    'priority': 'Unknown'
                    }
    
    try:
        parts = clean_text.split('|')
        if len(parts) == 3:
            parsed_data['district'] = parts[0].split(':', 1)[1].strip()
            parsed_data['intent']   = parts[1].split(':', 1)[1].strip()
            parsed_data['priority'] = parts[2].split(':', 1)[1].strip()

    except Exception as e:
        pass

    results.append(parsed_data)

    log_llm_call('openai', model, 'few_shot_classify', response['latency_ms'], response['usage'])
    
    print(f"{str(parsed_data['district']):<15} | {str(parsed_data['intent']):<15} | {str(parsed_data['priority']):<10} | {msg[:80]}...")

District        | Intent          | Priority   | Message                                           
--------------------------------------------------------------------------------
Colombo         | Info            | High       | BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critic...
Gampaha         | Rescue          | High       | SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boa...
Kandy           | Other           | Low        | Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims re...
Gampaha         | Supply          | High       | Does anyone have extra dry rations for the camp in Gampaha?...
Colombo         | Info            | Low        | News just in: Kelani river water level is at 7ft....
None            | Rescue          | High       | My uncle is stuck in the tree (immediate danger)....
Gampaha         | Info            | Low        | Just saw on news that Gampaha town is flooded. Hope eve

##### *Save to an Excel*

In [ ]:
df = pd.DataFrame(results)
output_path = f'{output_dir}/classified_messages.xlsx'

df.to_excel(output_path, index=False)
print(f"\nClassified messages saved to: {output_path}")
print(df[['original_message', 'intent', 'priority']].head())

##### *Success Check*

In [43]:
check_inputs = [
    "Breaking News: Kelani River level at 9m.",
    "We are trapped on the roof with 3 kids!"
]

for msg in check_inputs:
    prompt_text, spec = render(
                                'few_shot.v1',
                                role='Crisis Intelligence Classifier',
                                examples=examples,
                                query=f'Message: "{msg}"',
                                constraints='Classify the message exactly according to the Output format. Do not add extra text.',
                                format='District: [Name] | Intent: [Category] | Priority: [High/Low]'
                            )

    response = client.chat([{'role': 'user', 'content': prompt_text}], temperature=0.0)
    
    print(f"Input:  {msg}")
    print(f"Output: {response['text'].strip()}")

Input:  Breaking News: Kelani River level at 9m.
Output: District: Colombo | Intent: Info | Priority: Low
Input:  We are trapped on the roof with 3 kids!
Output: District: None | Intent: Rescue | Priority: High
